In [1]:
import os
import sys

In [2]:
#sys.path.insert(0, '/Users/marusy/Programming/another-fem-framework/')
#sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')
sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [3]:
import h5py
import numpy as np
from common import polynom_factory
from operator import itemgetter
import sympy
from common.tests import partial_diff_test

In [4]:
f_matrices = h5py.File('../datasources/1_10_globatto_matrices.h5')

In [8]:
f_lobatto_integrated_matrices = h5py.File('../datasources/1_10_globatto_integrated.h5')

In [9]:
orders_available = f_matrices['order_range'][()]

In [10]:
f_lobatto_integrated_matrices.create_group('M_maps')
for order in orders_available:
    f_lobatto_integrated_matrices['M_maps'].create_group('M_map_{}'.format(order))
    X, Y, indices = [f_matrices['M_maps/M_map_{}/{}'.format(order,x)][()] for x in ['X','Y', 'indices']]
    for s,x in zip(['X','Y', 'indices'],[X, Y, indices]):
        f_lobatto_integrated_matrices['M_maps/M_map_{}'.format(order)].create_dataset(s,data=x)
f_lobatto_integrated_matrices.create_dataset('order_range', data=orders_available)

<HDF5 dataset "order_range": shape (9,), type "<i4">

In [11]:
list(f_matrices.items())

[('M_maps', <HDF5 group "/M_maps" (9 members)>),
 ('grad_matrices', <HDF5 group "/grad_matrices" (9 members)>),
 ('mass_matrices', <HDF5 group "/mass_matrices" (9 members)>),
 ('order_range', <HDF5 dataset "order_range": shape (9,), type "<i4">)]

In [12]:
f_lobatto_integrated_matrices.create_group('mass_matrices')
f_lobatto_integrated_matrices.create_group('grad_matrices')

<HDF5 group "/grad_matrices" (0 members)>

In [13]:
for order in orders_available:
    mass_lobatto = f_matrices['mass_matrices/base_M_order_{}'.format(order)][()]
    grad_lobatto = np.zeros(mass_lobatto.shape)
    mass_lobatto = mass_lobatto.sum(axis=1)
    X, Y, indices = [f_matrices['M_maps/M_map_{}/{}'.format(order,x)][()] for x in ['X','Y', 'indices']]
    func_to_eval = polynom_factory.local_gradgrad_functions(dim=2, distribution='globatto', order=order)
    sorted_X = np.array([x for _,x in sorted(zip(indices,X))])
    sorted_Y = np.array([y for _,y in sorted(zip(indices,Y))])
    for ind, func in func_to_eval[0].items():
        grad_lobatto[ind] = np.sum(
            mass_lobatto * np.array(sympy.lambdify(modules='numpy',
                                                   args=func_to_eval[1],
                                                   expr=func)(sorted_X, sorted_Y))
        )
    f_lobatto_integrated_matrices['mass_matrices'].create_dataset(
        'base_M_order_{}'.format(order),
        data=np.diag(mass_lobatto)
    )
    f_lobatto_integrated_matrices['grad_matrices'].create_dataset(
        'M_order_{}'.format(order),
        data=grad_lobatto
    )
    print('\rorder {} done'.format(order), end='')
    

order 9 done

In [14]:
f_lobatto_integrated_matrices.close()